# Document Chracterizzation


## Import of relevant libraries 

In [34]:
# Import of all the libraries used for the project
import os
import shutil
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from operator import itemgetter
import plotly.graph_objects as go
import spacy


## Dataset import

In [2]:
# Defined a simple scraper  to go through all the link, visit the page and copy all the txt in the paragraph tag of the HTML file

def scraper(input):
    # create an empty array
    content = []

    # for loop on all the links in the dataframe
    for i in range(len(input["URL"])):
        url = input["URL"][i]
        # text = url
        text = ' '

        # check if actually the url is correct orr not
        if not (url.startswith('https://') or url.startswith('http://')):
            text += ' Link not good'
            content.append(text) 
            continue
        
        # Request the url and copy of the HTMl 
        page = requests.get(url)
        
        # Check if the response is positive or not, if not set that page was not found 
        if page.status_code != 200:
            text += ' Page not able to find'

        # Else take the text from all the p tag of the HTML content  
        else:
            soup = BeautifulSoup(page.content, 'html.parser')
            for p in soup.find_all('p'):
                text = text + ' ' + p.text
        
        content.append(text) 
    return content

In [3]:
## import the CSV file with the labels

data_scraper = pd.read_csv('Document Characterisation Data.csv', sep=',')
#print(len(data_scraper["URL"]))

# Calling the scraper function
data = scraper(data_scraper)

# Adding the scraped data into the Dataframe  
data_scraper['Content'] = data
data_scraper

,URL,Class,Content
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report,"2020-11-25 On 26 September, hundreds of Mali..."
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By George Barros November 16, 2020 The Kreml..."
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By Ezgi Yazici November 13, 2020 Key Takeawa..."
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report,By Katherine Lawlor with Brandon Wallace Key...
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report,By Isabel Ivanescu and Ezgi Yazici Key Takea...
...,...,...,...
232,https://www.journalism.org/2019/03/26/for-loca...,Analytical Report,"Numbers, Facts and Trends Shaping Your World..."
233,https://www.journalism.org/2017/10/02/covering...,Analytical Report,"Numbers, Facts and Trends Shaping Your World..."
234,https://ctc.usma.edu/afghanistans-future-emira...,Analytical Report,Main Menu Seth G. Jones \nShare via:\n\n\n\n...
235,https://ctc.usma.edu/back-into-the-shadows-the...,Analytical Report,Main Menu Michael Knights \nShare via:\n\n\n...


In [4]:
# Function to load the files from local drive into Pandas data frame

def readFiles (src, fileClass):
    src_files = os.listdir(src)
    fileType = []
    content = []
    for i in range(len(src_files)):
        fileType.append(fileClass)
        glove_file = open(src + src_files[i], "r", encoding="utf8")
        glove_vectors_list = [word_and_vector.strip() for word_and_vector in glove_file.readlines()]
        content.append(" ".join(glove_vectors_list))
    
    columns = ["URL", "Class", "Content"]
    df = pd.DataFrame(columns=columns)

    df["URL"] = src_files
    df["Class"] = fileType
    df["Content"] = content

    return df


In [5]:
src = "./Data/Analytical Report/"
df_Analytical_Report = readFiles(src, "Analytical Report")

print(len(df_Analytical_Report['Content'][0]))

src = "./Data/Analytical Report2/"
df_Analytical_Report2 = readFiles(src, "Analytical Report")

print(len(df_Analytical_Report2['Content'][0]))

src = "./Data/Incident Report/"
df_Incident_Report = readFiles(src, "Incident Report")

src = "./Data/Situation Report/"
df_Situation_Report = readFiles(src, "Situation Report")

toConcat = [data_scraper, df_Analytical_Report, df_Analytical_Report2, df_Incident_Report, df_Situation_Report]

data_input = pd.concat(toConcat, ignore_index=True)
data_input

5725
5725


,URL,Class,Content
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report,"2020-11-25 On 26 September, hundreds of Mali..."
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By George Barros November 16, 2020 The Kreml..."
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By Ezgi Yazici November 13, 2020 Key Takeawa..."
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report,By Katherine Lawlor with Brandon Wallace Key...
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report,By Isabel Ivanescu and Ezgi Yazici Key Takea...
...,...,...,...
413,int-rep-o-18-e.txt,Situation Report,Deputies are investigating after an explosive ...
414,int-rep-o-18-f.txt,Situation Report,Residents at a Springfield apartment complex w...
415,int-rep-o-18-g.txt,Situation Report,Several handheld explosive devices were found ...
416,int-rep-o-18-h.txt,Situation Report,Boulder police taped off the U.S. Bank at Broa...


## Experimental Data Analysis 

### Data frame informations and duplicates Duplicates

In [6]:
data_input.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   URL      418 non-null    object
 1   Class    418 non-null    object
 2   Content  418 non-null    object
dtypes: object(3)
memory usage: 9.9+ KB


In [7]:
# Number of items per class
print("Number of Analytical Report ", len(data_input[data_input['Class'] == 'Analytical Report']))
print("Number of Situation Report ", len(data_input[data_input['Class'] == 'Situation Report']))
print("Number of Incident Report ", len(data_input[data_input['Class'] == 'Incident Report']))
print("Number of Profile Report ", len(data_input[data_input['Class'] == 'Profile Report']))


Number of Analytical Report  131
Number of Situation Report  99
Number of Incident Report  88
Number of Profile Report  100


In [9]:
# Dropping the duplicate items 

data_input_dropped= data_input.drop_duplicates(keep="first").reset_index(drop=True)

In [10]:
# Number of items per class after dropping the duplicates
print("Number of Analytical Report ", len(data_input_dropped[data_input_dropped['Class'] == 'Analytical Report']))
print("Number of Situation Report ", len(data_input_dropped[data_input_dropped['Class'] == 'Situation Report']))
print("Number of Incident Report ", len(data_input_dropped[data_input_dropped['Class'] == 'Incident Report']))
print("Number of Profile Report ", len(data_input_dropped[data_input_dropped['Class'] == 'Profile Report']))


Number of Analytical Report  92
Number of Situation Report  98
Number of Incident Report  88
Number of Profile Report  98


In [11]:
data_input_dropped.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376 entries, 0 to 375
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   URL      376 non-null    object
 1   Class    376 non-null    object
 2   Content  376 non-null    object
dtypes: object(3)
memory usage: 8.9+ KB


### Pre Processing the data

In [12]:
# tockenizzation of the corpus 
def tokenize(corpus):

    # The backslash is an escape character to let the interpreter know we mean to use it as a string literal (- or ') 
    re_punctuation_string = '[\s,/.?\-\'`’]'  # split on spaces (\s),  commas (,), slash (/), fullstop (.), question marks (?), hyphens (-), and apostrophe (').
    tokenized = re.split(re_punctuation_string, corpus) # in python's regex, [...] is an alternative to writing .|.|.
    tokenized = list(filter(None, tokenized)) # remove empty strings from list 
    tokenized = [token.lower() for token in tokenized] # Lowercasing everything
    tokenized = ['<Number>' if token.isdigit() else token for token in tokenized ]  # converting all the numbers to <Number>


    return tokenized    



In [13]:
# function to count the tockenization
def distinct_tokens_count(tokenized_corpus):

    distinct_tokens_count = {}
    for t_review in tokenized_corpus:

        if t_review not in distinct_tokens_count.keys():
            distinct_tokens_count[t_review] = 1
        else:
            distinct_tokens_count[t_review] += 1

    # Sort the key value to greter to smaller
    distinct_tokens_count = sorted(distinct_tokens_count.items(), key=itemgetter(1), reverse=True) 

    #for token, count in distinct_tokens_count:
        #print("{:<14s} {:<10d}".format(token, count))
    return distinct_tokens_count



In [14]:
# function to pre process the text and count the number of words
def process(dataframe):

    tokenized = []
    tokens_count = []
    class_type = []
    for i in range(dataframe.shape[0]):
        #print(dataframe['Content'][i])
        #print(i)
        
        # tokenizing the corpus
        token = tokenize(dataframe['Content'][i])
        tokenized.append(token)
        # counting the tokens
        tokens_count.append(distinct_tokens_count(token))

        if dataframe['Class'][i] == 'Analytical Report':
            class_type.append(0)

        elif dataframe['Class'][i] == 'Incident Report':
            class_type.append(1)

        elif dataframe['Class'][i] == 'Profile Report':
            class_type.append(2)

        elif dataframe['Class'][i] == 'Situation Report':
            class_type.append(3)


    dataframe['Tokenized'] = tokenized
    dataframe['Tokens Count'] = tokens_count
    dataframe['Class Type'] = class_type

    return dataframe

In [15]:
data_processed = process(data_input_dropped)      

data_processed

,URL,Class,Content,Tokenized,Tokens Count,Class Type
0,https://issafrica.org/iss-today/time-to-tackle...,Analytical Report,"2020-11-25 On 26 September, hundreds of Mali...","[<Number>, <Number>, <Number>, on, <Number>, s...","[(the, 127), (and, 83), (women, 82), (to, 70),...",0
1,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By George Barros November 16, 2020 The Kreml...","[by, george, barros, november, <Number>, <Numb...","[(the, 67), (<Number>, 50), (to, 44), (csto, 3...",0
2,http://www.understandingwar.org/backgrounder/r...,Analytical Report,"By Ezgi Yazici November 13, 2020 Key Takeawa...","[by, ezgi, yazici, november, <Number>, <Number...","[(the, 41), (in, 25), (to, 24), (s, 22), (and,...",0
3,http://www.understandingwar.org/backgrounder/w...,Analytical Report,By Katherine Lawlor with Brandon Wallace Key...,"[by, katherine, lawlor, with, brandon, wallace...","[(s, 131), (the, 121), (to, 120), (<Number>, 1...",0
4,http://www.understandingwar.org/backgrounder/r...,Analytical Report,By Isabel Ivanescu and Ezgi Yazici Key Takea...,"[by, isabel, ivanescu, and, ezgi, yazici, key,...","[(in, 42), (the, 37), (and, 34), (a, 23), (to,...",0
...,...,...,...,...,...,...
371,int-rep-o-18-e.txt,Situation Report,Deputies are investigating after an explosive ...,"[deputies, are, investigating, after, an, expl...","[(the, 52), (a, 43), (was, 35), (to, 25), (dev...",3
372,int-rep-o-18-f.txt,Situation Report,Residents at a Springfield apartment complex w...,"[residents, at, a, springfield, apartment, com...","[(the, 64), (a, 47), (was, 27), (in, 22), (bom...",3
373,int-rep-o-18-g.txt,Situation Report,Several handheld explosive devices were found ...,"[several, handheld, explosive, devices, were, ...","[(the, 82), (a, 67), (bomb, 30), (to, 27), (wa...",3
374,int-rep-o-18-h.txt,Situation Report,Boulder police taped off the U.S. Bank at Broa...,"[boulder, police, taped, off, the, u, s, bank,...","[(the, 83), (a, 77), (was, 42), (to, 36), (bom...",3


### Feature extraction

In [16]:
def adding_features(dataframe):
    length = []
    unique_tokens = []
    tokens_percentage = []
    
    for i in range(dataframe.shape[0]):
        leng = len(data_processed['Tokenized'][i])
        length.append(leng)
        unique_tokens.append(len(data_processed['Tokens Count'][i]))

        distinct_tokens_percentage = {}
        for token, count in data_processed['Tokens Count'][i]:
            distinct_tokens_percentage[token] = (count * 100) / leng
        
        distinct_tokens_percentage = sorted(distinct_tokens_percentage.items(), key=itemgetter(1), reverse=True)
        tokens_percentage.append(distinct_tokens_percentage)
    
    
    dataframe['Length'] = length
    dataframe['Unique Tokens'] = unique_tokens
    dataframe['Tokens Percentage'] = tokens_percentage
    # tahen out all the page not accessible with message "Page not found"
    dataframe = dataframe[dataframe['Length'] > 10].reset_index(drop=True)

    return dataframe

In [17]:
data_add_features = adding_features(data_processed)

data_Analytical = data_add_features[data_add_features['Class'] == 'Analytical Report']
data_Incident = data_add_features[data_add_features['Class'] == 'Incident Report']
data_Profile = data_add_features[data_add_features['Class'] == 'Profile Report']
data_Situation = data_add_features[data_add_features['Class'] == 'Situation Report']

In [31]:
# Function to plot the violin plot for the dataframe   
def plot_violin(df, to_be_plot):

    fig = go.Figure()

    reports = ['Analytical Report', 'Incident Report', 'Profile Report', 'Situation Report']

    for report in reports:
        fig.add_trace(go.Violin(x=df['Class'][df['Class'] == report],
                                y=df[to_be_plot][df['Class'] == report],
                                name=report,
                                box_visible=True,
                                meanline_visible=True))
    
    fig.update_layout(
    title="Violin plot of the " + to_be_plot + " for type of reports",
    yaxis_range=[0, 5000],
    )
    fig.show()

    return


In [32]:
plot_violin(data_add_features, "Length")
plot_violin(data_add_features, "Unique Tokens")

In [42]:

nlp = spacy.load("en_core_web_sm")
doc = nlp(data_add_features['Content'][0])

for token in doc:
    print(token.text, token.pos_,  token.dep_)



   SPACE 
2020 NUM ROOT
- PUNCT punct
11 NUM nummod
- SYM punct
25 NUM prep
On ADP prep
26 NUM nummod
September PROPN pobj
, PUNCT punct
hundreds NOUN nsubj
of ADP prep
Malian ADJ amod
women NOUN pobj
and CCONJ cc
some DET det
men NOUN nsubj
rallied VERB ROOT
in ADP prep
Bamako PROPN pobj
to PART aux
protest VERB xcomp
against ADP prep
the DET det
trivialisation NOUN pobj
of ADP prep
violence NOUN pobj
against ADP prep
women NOUN pobj
. PUNCT punct
This DET nsubjpass
was AUX auxpass
prompted VERB ROOT
by ADP agent
the DET det
revelation NOUN pobj
on ADP prep
social ADJ amod
media NOUN pobj
of ADP prep
alleged ADJ amod
intimate ADJ amod
partner NOUN compound
abuse NOUN pobj
involving VERB acl
a DET det
famous ADJ amod
Malian ADJ amod
artist NOUN dobj
. PUNCT punct

 SPACE 
The DET det
story NOUN nsubj
broke VERB ROOT
just ADV advmod
two NUM nummod
months NOUN npadvmod
before ADP prep
the DET det
25 NUM nummod
November PROPN nmod
annual ADJ amod
international ADJ amod
day NOUN pobj
for A

Pre trained BERT models

https://pypi.org/project/pytorch-pretrained-bert/

https://pytorch.org/hub/huggingface_pytorch-transformers/

https://medium.com/@aniruddha.choudhury94/part-2-bert-fine-tuning-tutorial-with-pytorch-for-text-classification-on-the-corpus-of-linguistic-18057ce330e1


Spacy linguistic features
https://spacy.io/usage/linguistic-features
